In [ ]:
import unicodedata
import pandas as pd
import numpy as np


lista = ['        Novais Barros Ltda.', 'Freitas Monteiro          Ltda.', 'Abreu         Cirino                Ltda.']

df = pd.DataFrame({
        'teste':   lista,
        'coluna2': [1,2,3]
})

#remove espacos desnecessarios antes, entre e depois dos valores em formato string
df = df(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

import numpy as np

# Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Converte a coluna para string, remove espaços extras e aplica strip
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    # Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.datetime64):  # Se for data
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped

# Aplica a função a todas as colunas do DataFrame
df = df.apply(strip_all_types)

lista_formatada = []

# Processa cada item da lista
for item in lista:
    #espacos desnecessarios antes e depois da string
    item = item.strip()
    # Remove acentos
    item_sem_acentos = ''.join(
        letra for letra in unicodedata.normalize('NFD', item)
        if unicodedata.category(letra) != 'Mn'
    )
    
    # Remove caracteres especiais e pontos
    item_formatado = ''.join(
        letra for letra in item_sem_acentos
        if letra.isalnum() or letra.isspace()
    )
    
    # Adiciona o item formatado à lista
    lista_formatada.append(item_formatado)

# Exibe a lista formatada
display(df)


,teste,coluna2
0,Novais Barros Ltda.,1
1,Freitas Monteiro Ltda.,2
2,Abreu Cirino Ltda.,3
